In [1]:
from mxnet import gluon,init,nd
from mxnet.gluon import nn

In [2]:
net = nn.Sequential()
net.add(nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),
        nn.MaxPool2D(pool_size=3,strides=2),
        nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
        nn.MaxPool2D(pool_size=3,strides=2),
        nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
        nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
        nn.Conv2D(256,kernel_size=3,padding=1,activation='relu'),
        nn.MaxPool2D(pool_size=3,strides=2),
        nn.Flatten(),#构成一个全连接
        nn.Dense(4096,activation='relu'),
        nn.Dropout(0.5),#防止过拟合
        nn.Dense(4096,activation='relu'),
        nn.Dropout(0.5),
        nn.Dense(10))

In [3]:
X = nd.random_uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv0 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
conv1 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
conv2 output shape:	 (1, 384, 12, 12)
conv3 output shape:	 (1, 384, 12, 12)
conv4 output shape:	 (1, 256, 12, 12)
pool2 output shape:	 (1, 256, 5, 5)
flatten0 output shape:	 (1, 6400)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)


In [4]:
import utils
from mxnet import image
batch_size = 128
train_data,test_data = utils.load_data_fashion_mnist(batch_size,resize=(224,224))

In [5]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [6]:
def accuracy(output,label):
    return nd.mean(output.argmax(axis = 1) == label).asscalar()

In [7]:
def evaluate_accuracy(data_iterator,net):
    acc = 0
    for data,label in data_iterator:
        output = net(data)
        acc += accuracy(output,label)
    return acc/len(data_iterator)

In [10]:
from mxnet import autograd
lr,num_epochs = 0.01, 5
net.initialize(init=init.Xavier(),force_reinit=True)
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
for e in range(num_epochs):
    train_loss = 0
    train_acc = 0
    test_acc = 0
    for data,label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output,label)
        loss.backward()
        trainer.step(batch_size)
        train_loss += nd.mean(loss).asscalar()
#         train_acc += accuracy(output,label)
#     test_acc = utils.evaluate_accuracy(test_data,net)
    print("Epoch: %d,train_loss: %f,train_acc: %f, test_acc: %f" % (e,train_loss/len(train_data),train_acc/len(train_data),test_acc))

Epoch: 0,train_loss: 1.267184,train_acc: 0.000000, test_acc: 0.000000
Epoch: 1,train_loss: 0.645976,train_acc: 0.000000, test_acc: 0.000000
Epoch: 2,train_loss: 0.528928,train_acc: 0.000000, test_acc: 0.000000
Epoch: 3,train_loss: 0.465504,train_acc: 0.000000, test_acc: 0.000000
Epoch: 4,train_loss: 0.421147,train_acc: 0.000000, test_acc: 0.000000
